# Model Speed

## Benchmarking the Fit and Predict Speed of the Model

1. Benchmarking takes place where each measurement takes 10 rounds.
2. Average and Worst times will be recorded.
3. Model Performance is not the focus here but rather its speed.

In [1]:
import pandas as pd
import json
import time
import lightgbm
import catboost
from sklearn.model_selection import train_test_split

from joblib import load
import warnings
warnings.filterwarnings("ignore")

def parse_tune(tune_file:str):
    f = open(tune_file, "r")
    params = f.readline()
    f.close()
    params = params.replace('\'', '\"')#.replace("False", "\"False\"").replace("True", "\"True\"")
    params = json.loads(str(params))
    print(params)
    return params

def get_indexes():
    indexes = []
    for i in range(100):
        indexes.append(f"t_{i}")
    return indexes

LOOPS = 10 # Each benchmark takes 10 rounds per model/behavior-type

KeyboardInterrupt: 

# 1 Fit Speed

In [ ]:
def fit_benchmark(model,x,y,title:str,loops:int):
    times = []
    for i in range(1,loops+1):
        print(f"{title} Round: {i}", end="\r")
        start = time.time()
        model.fit(x,y)
        times.append(time.time()-start)
    print(f"{title}")
    print(f"Best Time: {min(times):.4f}s")
    print(f"Average Time: {sum(times)/len(times):.4f}s")
    print(f"Worst Time: {max(times):.4f}s")

In [ ]:
train = pd.read_csv('../Dataset/TB/LGBM_TB.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
X_train, X_valid, y_train, y_valid = train_test_split(train.iloc[:,1:101], train.iloc[:,0], test_size=.3, shuffle=True, random_state=1)

#LGBM TB
times = []
lgbm_tb = lightgbm.LGBMClassifier(random_state=1, n_jobs=0, verbose=-1)
fit_benchmark(lgbm_tb, X_train, y_train, "Default LGBM_TB", LOOPS)
print("")

#LGBM TB (Tuned)
tuning = parse_tune('../GBDT_Training/Outputs/LGBM/Tuning/Manual/RYZEN3b_LGBM_TB.tune')
tuned_lgbm_tb = lightgbm.LGBMClassifier(**tuning, random_state=1, n_jobs=0, verbose=-1,)
fit_benchmark(tuned_lgbm_tb,X_train,y_train, "Tuned LGBM_TB", LOOPS)

In [ ]:
train = pd.read_csv('../Dataset/IB/LGBM_IB.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
X_train, X_valid, y_train, y_valid = train_test_split(train.iloc[:,1:101], train.iloc[:,0], test_size=.3, shuffle=True, random_state=1)

#LGBM IB
times = []
lgbm_ib = lightgbm.LGBMClassifier(random_state=1, n_jobs=0, verbose=-1)
fit_benchmark(lgbm_ib, X_train, y_train, "Default LGBM_IB", LOOPS)
print("")

#LGBM IB (Tuned)
tuning = parse_tune('../GBDT_Training/Outputs/LGBM/Tuning/Manual/RYZEN3b_LGBM_IB.tune')
tuned_lgbm_ib = lightgbm.LGBMClassifier(**tuning, random_state=1, n_jobs=0, verbose=-1,)
fit_benchmark(tuned_lgbm_ib,X_train,y_train, "Tuned LGBM_IB", LOOPS)

In [ ]:
train = pd.read_csv('../Dataset/TB/CATB_TB.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
X_train, X_valid, y_train, y_valid = train_test_split(train.iloc[:,1:101], train.iloc[:,0], test_size=.3, shuffle=True, random_state=1)

#CATB TB
times = []
catb_tb = catboost.CatBoostClassifier(random_state=1, thread_count=-1, verbose=0, cat_features=get_indexes(), nan_mode='Min', one_hot_max_size=256)
fit_benchmark(catb_tb, X_train, y_train, "Default CATB_TB", LOOPS)
print("")

# #CATB TB (Tuned)
tuning = parse_tune('../GBDT_Training/Outputs/CATB/Tuning/Manual/RYZEN3b_CATB_TB.tune')
tuned_catb_tb = catboost.CatBoostClassifier(**tuning, random_state=1, thread_count=-1, verbose=0, cat_features=get_indexes(), nan_mode='Min', one_hot_max_size=256)
fit_benchmark(tuned_catb_tb,X_train,y_train, "Tuned CATB_TB", LOOPS)

In [ ]:
train = pd.read_csv('../Dataset/IB/CATB_IB.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
train.iloc[:,1:101] = train.iloc[:,1:101].astype('str')
train.replace("nan", "NaN", inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(train.iloc[:,1:101], train.iloc[:,0], test_size=.3, shuffle=True, random_state=1)

#CATB IB
times = []
catb_ib = catboost.CatBoostClassifier(random_state=1, thread_count=-1, verbose=0, cat_features=get_indexes(), nan_mode='Min', one_hot_max_size=256)
fit_benchmark(catb_ib, X_train, y_train, "Default CATB_IB", LOOPS)
print("")

#CATB IB (Tuned)
tuning = parse_tune('../GBDT_Training/Outputs/CATB/Tuning/Manual/RYZEN3b_CATB_IB.tune')
tuned_catb_ib = catboost.CatBoostClassifier(**tuning, random_state=1, thread_count=-1, verbose=0, cat_features=get_indexes(), nan_mode='Min', one_hot_max_size=256)
fit_benchmark(tuned_catb_ib,X_train,y_train, "Tuned CATB_IB", LOOPS)

# 2 Predict Speed

In [ ]:
DF_LGBM_TB = pd.read_csv('../Dataset/TB/LGBM_TB_Test.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
DF_LGBM_IB = pd.read_csv('../Dataset/IB/LGBM_IB_Test.csv', low_memory=False)
DF_CATB_TB = pd.read_csv('../Dataset/TB/CATB_TB_Test.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
DF_CATB_IB = pd.read_csv('../Dataset/IB/CATB_IB_Test.csv', low_memory=False)
DF_CATB_IB.iloc[:,1:101] = DF_CATB_IB.iloc[:,1:101].astype('str')
DF_CATB_IB.replace("nan", "NaN", inplace=True)

def pred_benchmark(model,x,y,title:str,loops:int):
    times = []
    for i in range(1,loops+1):
        print(f"{title} Round: {i}", end="\r")
        start = time.time()
        model.predict(x)
        times.append(time.time()-start)
    print(f"{title}")
    print(f"Best Time: {min(times)/x.size*1e+9:.4f}ns")
    print(f"Average Time: {sum(times)/len(times)/x.size*1e+9:.4f}ns")
    print(f"Worst Time: {max(times)/x.size*1e+9:.4f}ns")

In [ ]:
pred_benchmark(lgbm_tb, DF_LGBM_TB.iloc[:,1:101], DF_LGBM_TB.iloc[:,0], "Default LightGBM TB", LOOPS)
print("")

pred_benchmark(tuned_lgbm_tb, DF_LGBM_TB.iloc[:,1:101], DF_LGBM_TB.iloc[:,0], "Tuned LightGBM TB", LOOPS)

In [ ]:
pred_benchmark(lgbm_ib, DF_LGBM_IB.iloc[:,1:101], DF_LGBM_IB.iloc[:,0], "Default LightGBM IB", LOOPS)
print("")

pred_benchmark(tuned_lgbm_ib, DF_LGBM_IB.iloc[:,1:101], DF_LGBM_IB.iloc[:,0], "Tuned LightGBM IB", LOOPS)

In [ ]:
pred_benchmark(catb_tb, DF_CATB_TB.iloc[:,1:101], DF_CATB_TB.iloc[:,0], "Default CatBoost TB", LOOPS)
print("")

pred_benchmark(tuned_catb_tb, DF_CATB_TB.iloc[:,1:101], DF_CATB_TB.iloc[:,0], "Tuned CatBoost TB", LOOPS)

In [ ]:
pred_benchmark(catb_ib, DF_CATB_IB.iloc[:,1:101], DF_CATB_IB.iloc[:,0], "Default CatBoost IB", LOOPS)
print("")

pred_benchmark(tuned_catb_ib, DF_CATB_IB.iloc[:,1:101], DF_CATB_IB.iloc[:,0], "Tuned CatBoost IB", LOOPS)